In [8]:
%load_ext autoreload
%autoreload 2


"""
Author: Ashyam Zubair
Created Date: 14-02-2019
"""
import json
import pandas as pd
import traceback
import sqlalchemy
import os
import pyodbc
import sys
import pymssql
import numpy as np

import logging

from MySQLdb._exceptions import OperationalError
from sqlalchemy import create_engine, exc
from time import time

#connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=192.168.15.126;UID=BRS;PWD=Fint$123;Trusted Connection=yes;DATABASE="
connection_string = None
#try:
#    from app.ace_logger import Logging
#except:
#    from ace_logger import Logging
#    pass
    
#logging = Logging()

class DB(object):
    def __init__(self, database, host='127.0.0.1', user='root', password='', port='3306', tenant_id=None):
        """
        Initialization of databse object.
        Args:
            databse (str): The database to connect to.
            host (str): Host IP address. For dockerized app, it is the name of
                the service set in the compose file.
            user (str): Username of MySQL server. (default = 'root')
            password (str): Password of MySQL server. For dockerized app, the
                password is set in the compose file. (default = '')
            port (str): Port number for MySQL. For dockerized app, the port that
                is mapped in the compose file. (default = '3306')
        """

        if host in ["common_db","extraction_db", "queue_db", "template_db", "table_db", "stats_db", "business_rules_db", "reports_db"]:
            self.HOST = os.environ['HOST_IP']
            self.USER = 'sa'
            self.PASSWORD = os.environ['LOCAL_DB_PASSWORD']
            self.PORT = os.environ['LOCAL_DB_PORT']
            self.DATABASE = f'{tenant_id}_{database}' if tenant_id is not None and tenant_id else database
        else:
            self.HOST = os.environ['HOST_IP']
            self.USER = 'sa'
            self.PASSWORD = os.environ['LOCAL_DB_PASSWORD']
            self.PORT = os.environ['LOCAL_DB_PORT']
            self.DATABASE = f'{tenant_id}_{database}' if tenant_id is not None and tenant_id else database
        
        logging.info(f'Host: {self.HOST}')
        logging.info(f'User: {self.USER}')
        logging.info(f'Password: {self.PASSWORD}')
        logging.info(f'Port: {self.PORT}')
        logging.info(f'Database: {self.DATABASE}')
        # self.connect()
    def connect(self, max_retry=5):
#         retry = 1

#         try:
#             start = time()
#             logging.debug(f'Making connection to {self.DATABASE}...')
#             config = f'mysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}?charset=utf8'
#             self.db_ = create_engine(config, connect_args={'connect_timeout': 2}, pool_recycle=300)
#             logging.info(f'Engine created for {self.DATABASE}')
#             while retry <= max_retry:
#                 try:
#                     self.engine = self.db_.connect()
#                     logging.info(f'Connection established succesfully to {self.DATABASE}! ({round(time() - start, 2)} secs to connect)')
#                     break
#                 except Exception as e:
#                     logging.warning(f'Connection failed. Retrying... ({retry}) [{e}]')
#                     retry += 1
#                     self.db_.dispose()
#         except:
#             logging.exception(f'Something went wrong while connecting. Check trace.')
#             return
        data = []
        inds = [i for i in range(len(sql)) if sql[i] == '']
        print(inds)
        for pos, ind in enumerate(inds):
            if pos % 2 == 0:
                sql = sql[:ind] + '[' + sql[ind+1:]
            else:
                sql = sql[:ind] + ']' + sql[ind + 1:]
        if connection_string:
            try:
                conn = pyodbc.connect(connection_string + self.DATABASE)
            except Exception as e:
                print('Connection string invalid. ', e)
        else:
            try:
                if user_ or password_:
                    conn = pyodbc.connect('DRIVER={' + driver + '};SERVER=' + host_ + ';DATABASE=' + database+ ';UID=' + user_ + ';PWD=' + password_ + ';Trusted Connection=yes;')
                else:
                    conn = pyodbc.connect('DRIVER={' + driver + '};SERVER=' + host_ + ';DATABASE=' + database + ';Trusted Connection=yes;')
            except Exception as e:
                print("Error establishing connection to DB. ", e)
                conn = pyodbc.connect('DRIVER={' + driver + '};SERVER=' + host_ + ';DATABASE=' + database + ';Trusted Connection=yes;')

    def convert_to_mssql(self, query):
        inds = [i for i in range(len(query)) if query[i] == '`']
        for pos, ind in enumerate(inds):
            if pos % 2 == 0:
                query = query[:ind] + '[' + query[ind+1:]
            else:
                query = query[:ind] + ']' + query[ind + 1:]
        
        query = query.replace('%s', '?')

        return query

    def execute(self, query, database=None, index_col='id', **kwargs):
        logging.debug(f'Before converting: {query}')
        query = self.convert_to_mssql(query)
        logging.debug(f'After converting: {query}')

        logging.debug('Connecting to DB')
        conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={self.HOST};UID={self.USER};PWD={self.PASSWORD};Trusted Connection=yes;DATABASE={self.DATABASE}', as_dict=True)
        logging.debug(f'Connection established with {self.DATABASE}. [{conn}]')
        curs = conn.cursor()
        logging.debug(f'Cursor object created. [{curs}]')
        params = tuple(kwargs.get('params', []))
        
        logging.debug(f'Params: {params}')
        logging.debug(f'Params Type: {type(params)}')
        params = [int(i) if isinstance(i, np.int64) else i for i in params]
        curs.execute(query, params)
        logging.debug(f'Query executed.')
        
        data = None

        try:
            logging.debug(f'Fetching all data.')
            data = curs.fetchall()
            # logging.debug(f'Data fetched: {data}')
            columns = [column[0] for column in curs.description]
            logging.debug(f'Columns: {columns}')
            result = []
            for row in data:
                result.append(dict(zip(columns, row)))
            # logging.debug(f'Zipped result: {result}')
            if result:
                data = pd.DataFrame(result)
            else:
                data = pd.DataFrame(columns=columns)
            # logging.debug(f'Data to DF: {data}')
        except:
            logging.debug('Update Query')
        conn.commit()
        conn.close()
        if not isinstance(data, pd.DataFrame):
            logging.debug(f'Data is not a DataFrame. Returning True. [{type(data)}]')
            return True
        
        try:
            if index_col is not None:
                logging.debug(f'Setting ID as index')
                return data.where((pd.notnull(data)), None).set_index('id')
            else:
                return data.where((pd.notnull(data)), None)
        except:
            logging.exception(f'Failed to set ID as index')
            return data.where((pd.notnull(data)), None)

    def execute__(self, query, database=None, **kwargs):
        """
        Executes an SQL query.
        Args:
            query (str): The query that needs to be executed.
            database (str): Name of the database to execute the query in. Leave
                it none if you want use database during object creation.
            params (list/tuple/dict): List of parameters to pass to in the query.
        Returns:
            (DataFrame) A pandas dataframe containing the data from the executed
            query. (None if an error occurs)
        """
        data = None

#         # Use new database if a new databse is given
#         if database is not None:
#             try:
#                 config = f'mysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{database}?charset=utf8'
#                 engine = create_engine(config, pool_recycle=300)
#             except:
#                 logging.exception(f'Something went wrong while connecting. Check trace.')
#                 return False
#         else:
#             engine = self.engine
        
        print('query', query)
        if database is None:
            database = 'karvy'
        data = None
        sql = query
        user_ = self.USER
        host_ = self.HOST
        database = self.DATABASE
        password_ = self.PASSWORD
        inds = [i for i in range(len(sql)) if sql[i] == '']
        for pos, ind in enumerate(inds):
            if pos % 2 == 0:
                sql = sql[:ind] + '[' + sql[ind+1:]
            else:
                sql = sql[:ind] + ']' + sql[ind + 1:]
               
        if connection_string:
            print('connection string', connection_string)
            print('database', database)
            print(type(connection_string + database))
            print(type(connection_string + database))

            try:
                conn = pyodbc.connect(connection_string + database)
            except Exception as e:
                print('Connection string invalid. ', e)
        else:
            try:
                if user_ or password_:
                    conn = pymssql.connect(host=host_,database=database,user=user_,password=password_)
                else:
                    conn = pymssql.connect(host=host_,database=database)
            except Exception as e:
                print("Error establishing connection to DB. ", e)
                conn = pymssql.connect(host=host_,database=database)
        try:
            logging.debug(f'Query: {query}')
            logging.debug(f'Query: {query}')
            curs = conn.cursor(as_dict = True)
            params = kwargs.get('params', [])
            logging.debug(f'Params: {params}')
            curs.execute(sql, tuple(params))
            print('query executed')
            try:
                data = curs.fetchall()
                data = pd.DataFrame(data)
                # print(data)
            except Exception as e:
                logging.debug('Update Query')
            # data = pd.read_sql(sql, conn, index_col='id', **kwargs)
        except exc.ResourceClosedError:
            logging.warning('Query does not have any value to return.')
            return True
        except (exc.StatementError, OperationalError) as e:
            logging.warning(f'Creating new connection. Engine/Connection is probably None. [{e}]')
            self.connect()
            data = pd.read_sql(query, self.engine, index_col='id', **kwargs)
        except:
            logging.exception('Something went wrong executing query. Check trace.')
            params = kwargs['params'] if 'params' in kwargs else None
            conn.rollback() 
            return False
        conn.close()
        return data.where((pd.notnull(data)), None).set_index('id')

    def execute_(self, query, database=None, **kwargs):
        """
        Executes an SQL query.
        Args:
            query (str): The query that needs to be executed.
            database (str): Name of the database to execute the query in. Leave
                it none if you want use database during object creation.
            params (list/tuple/dict): List of parameters to pass to in the query.
        Returns:
            (DataFrame) A pandas dataframe containing the data from the executed
            query. (None if an error occurs)
        """
        logging.debug(f'Executing `execute` instead of `execute_`')
        return self.execute(query, index_col=None, **kwargs)
        
        data = None

#         # Use new database if a new database is given
#         if database is not None:
#             try:
#                 config = f'mysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{database}?charset=utf8'
#                 engine = create_engine(config, pool_recycle=300)
#             except:
#                 logging.exception(f'Something went wrong while connecting. Check trace.')
#                 return False
#         else:
#             engine = self.engine

#         try:
#             data = pd.read_sql(query, engine, **kwargs)
#         except exc.ResourceClosedError:
#             return True
#         except:
#             logging.exception(f'Something went wrong while connecting. Check trace.')
#             params = kwargs['params'] if 'params' in kwargs else None
#             return False
        print('query', query)
        if database is None:
            database = 'karvy'
        data = None
        sql = query
        user_ = self.USER
        database = self.DATABASE
        host_ = self.HOST
        password_ = self.PASSWORD
        inds = [i for i in range(len(sql)) if sql[i] == '']
        # for pos, ind in enumerate(inds):
        #     if pos % 2 == 0:
        #         sql = sql[:ind] + '[' + sql[ind+1:]
        #     else:
        #         sql = sql[:ind] + ']' + sql[ind + 1:]
               
        if connection_string:
            print('connection string', connection_string)
            print('database', database)
            print(type(connection_string + database))
            print(type(connection_string + database))

            try:
                conn = pyodbc.connect(connection_string + database)
            except Exception as e:
                print('Connection string invalid. ', e)
        else:
            try:
                if user_ or password_:
                    conn = pymssql.connect(host=host_,database=database,user=user_,password=password_)
                else:
                    conn = pymssql.connect(host=host_,database=database)
            except Exception as e:
                print("Error establishing connection to DB. ", e)
                conn = pymssql.connect(host=host_,database=database)
        try:
            logging.debug(f'Query: {query}')
            curs = conn.cursor(as_dict = True)
            params = kwargs.get('params', [])
            logging.debug(f'Params: {params}')
            curs.execute(sql, params)
            print('query executed')
            try:
                data = curs.fetchall()
                data = pd.DataFrame(data)
                print(data)
            except Exception as e:
                logging.debug('Update Query')

            #data = pd.read_sql(sql, conn,**kwargs)
        except exc.ResourceClosedError:
            logging.warning('Query does not have any value to return.')
            return True
        except (exc.StatementError, OperationalError) as e:
            logging.warning(f'Creating new connection. Engine/Connection is probably None. [{e}]')
            self.connect()
            data = pd.read_sql(query, conn,**kwargs)
        except:
            logging.exception('Something went wrong executing query. Check trace.')
            params = kwargs['params'] if 'params' in kwargs else None
            conn.rollback() 
            return False
        conn.commit()
        conn.close()
#         return data.where((pd.notnull(data)), None)
        try:
            return data.replace({pd.np.nan: None}).set_index('id')
        except AttributeError as e:
            return True


    def insert(self, data, table, database=None, **kwargs):
        """
        Write records stored in a DataFrame to a SQL database.
        Args:
            data (DataFrame): The DataFrame that needs to be write to SQL database.
            table (str): The table in which the rcords should be written to.
            database (str): The database the table lies in. Leave it none if you
                want use database during object creation.
            kwargs: Keyword arguments for pandas to_sql function.
                See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
                to know the arguments that can be passed.
        Returns:
            (bool) True is succesfully inserted, else false.
        """
        logging.info(f'Inserting into {table}')

        # # Use new database if a new databse is given
        # if database is not None:
        #     try:
        #         config = f'mysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{database}?charset=utf8'
        #         engine = create_engine(config, pool_recycle=300)
        #     except:
        #         logging.exception(f'Something went wrong while connecting. Check trace.')
        #         return False
        # else:
        #     engine = self.engine

        print('query', query)
        if database is None:
            database = 'karvy'
        data = None
        sql = query
        user_ = self.USER
        database = self.DATABASE
        host_ = self.HOST
        password_ = self.PASSWORD
        inds = [i for i in range(len(sql)) if sql[i] == '']
        # for pos, ind in enumerate(inds):
        #     if pos % 2 == 0:
        #         sql = sql[:ind] + '[' + sql[ind+1:]
        #     else:
        #         sql = sql[:ind] + ']' + sql[ind + 1:]
               
        if connection_string:
            print('connection string', connection_string)
            print('database', database)
            print(type(connection_string + database))
            print(type(connection_string + database))

            try:
                conn = pyodbc.connect(connection_string + database)
            except Exception as e:
                print('Connection string invalid. ', e)
        else:
            try:
                if user_ or password_:
                    conn = pymssql.connect(host=host_,database=database,user=user_,password=password_)
                else:
                    conn = pymssql.connect(host=host_,database=database)
            except Exception as e:
                print("Error establishing connection to DB. ", e)
                conn = pymssql.connect(host=host_,database=database)
        try:
            logging.debug(f'Query: {query}')
            # data.to_sql(table, conn, **kwargs)
            curs = conn.cursor(as_dict = True)
            curs.execute(sql, tuple(kwargs.get('params', [])))
            print('query executed')
            try:
                data = curs.fetchall()
                data = pd.DataFrame(data)
                print(data)
            except Exception as e:
                logging.debug('Update Query')

            try:
                self.execute(f'ALTER TABLE {table} ADD PRIMARY KEY (id);')
            except:
                pass
            conn.commit()
            conn.close()
            return True
        except:
            logging.exception('Something went wrong inserting. Check trace.')
            return False

    def insert_dict(self, data, table):
        """
        Insert dictionary into a SQL database table.
        Args:
            data (DataFrame): The DataFrame that needs to be write to SQL database.
            table (str): The table in which the rcords should be written to.
        Returns:
            (bool) True is succesfully inserted, else false.
        """
        logging.info(f'Inserting dictionary data into {table}...')
        logging.debug(f'Data:\n{data}')

        try:
            column_names = []
            params = []

            for column_name, value in data.items():
                column_names.append(f'{column_name}')
                params.append(value)

            logging.debug(f'Column names: {column_names}')
            logging.debug(f'Params: {params}')

            columns_string = ', '.join(column_names)
            param_placeholders = ', '.join(['%s'] * len(column_names))

            query = f'INSERT INTO {table} ({columns_string}) VALUES ({param_placeholders})'

            return self.execute(query, params=params)
        except:
            logging.exception('Error inserting data.')
            return False

    def update(self, table, update=None, where=None, database=None, force_update=False):
        # Use new database if a new databse is given
        # if database is not None:
        #     try:
        #         config = f'mysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{database}?charset=utf8'
        #         self.engine = create_engine(config, pool_recycle=300)
        #     except:
        #         logging.exception(f'Something went wrong while connecting. Check trace.')
        #         return False

        logging.info(f'Updating table: {table}')
        logging.info(f'Update data: {update}')
        logging.info(f'Where clause data: {where}')
        logging.info(f'Force update flag: {force_update}')

        try:
            set_clause = []
            set_value_list = []
            where_clause = []
            where_value_list = []

            if where is not None and where:
                for set_column, set_value in update.items():
                    set_clause.append(f'{set_column}=%s')
                    set_value_list.append(set_value)
                set_clause_string = ', '.join(set_clause)
            else:
                logging.error(f'Update dictionary is None/empty. Must have some update clause.')
                return False

            if where is not None and where:
                for where_column, where_value in where.items():
                    where_clause.append(f'{where_column}=%s')
                    where_value_list.append(where_value)
                where_clause_string = ' AND '.join(where_clause)
                query = f'UPDATE {table} SET {set_clause_string} WHERE {where_clause_string}'
            else:
                if force_update:
                    query = f'UPDATE {table} SET {set_clause_string}'
                else:
                    message = 'Where dictionary is None/empty. If you want to force update every row, pass force_update as True.'
                    logging.error(message)
                    return False

            params = set_value_list + where_value_list
            self.execute(query, params=params)
            return True
        except:
            logging.exception('Something went wrong updating. Check trace.')
            return False

    def get_column_names(self, table, database=None):
        """
        Get all column names from an SQL table.
        Args:
            table (str): Name of the table from which column names should be extracted.
            database (str): Name of the database in which the table lies. Leave
                it none if you want use database during object creation.
        Returns:
            (list) List of headers. (None if an error occurs)
        """
        try:
            logging.info(f'Getting column names of table {table}')
            return list(self.execute(f'SELECT * FROM {table}', database))
        except:
            logging.exception('Something went wrong getting column names. Check trace.')
            return

    def execute_default_index(self, query, database=None, **kwargs):
        """
        Executes an SQL query.
        Args:
            query (str): The query that needs to be executed.
            database (str): Name of the database to execute the query in. Leave
                it none if you want use database during object creation.
            params (list/tuple/dict): List of parameters to pass to in the query.
        Returns:
            (DataFrame) A pandas dataframe containing the data from the executed
            query. (None if an error occurs)
        """

        logging.debug(f'Executing `execute` instead of `execute_default_index`')
        return self.execute(query, index_col=None, **kwargs)
        data = None

        # # Use new database if a new databse is given
        # if database is not None:
        #     try:
        #         config = f'mysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{database}?charset=utf8'
        #         engine = create_engine(config, pool_recycle=300)
        #     except:
        #         logging.exception(f'Something went wrong while connecting. Check trace.')
        #         return False
        # else:
        #     engine = self.engine

        print('query', query)
        if database is None:
            database = 'karvy'
        data = None
        sql = query
        user_ = self.USER
        host_ = self.HOST
        database = self.DATABASE
        password_ = self.PASSWORD
        inds = [i for i in range(len(sql)) if sql[i] == '']
        for pos, ind in enumerate(inds):
            if pos % 2 == 0:
                sql = sql[:ind] + '[' + sql[ind+1:]
            else:
                sql = sql[:ind] + ']' + sql[ind + 1:]
               
        if connection_string:
            print('connection string', connection_string)
            print('database', database)
            print(type(connection_string + database))
            print(type(connection_string + database))

            try:
                conn = pyodbc.connect(connection_string + database)
            except Exception as e:
                print('Connection string invalid. ', e)
        else:
            try:
                if user_ or password_:
                    conn = pymssql.connect(host=host_,database=database,user=user_,password=password_)
                else:
                    conn = pymssql.connect(host=host_,database=database)
            except Exception as e:
                print("Error establishing connection to DB. ", e)
                conn = pymssql.connect(host=host_,database=database)

        try:
            logging.debug(f'Query: {query}')
            # data.to_sql(table, conn, **kwargs)
            curs = conn.cursor(as_dict = True)
            
            curs.execute(sql, tuple(kwargs.get('params', [])))
            print('query executed')
            try:
                data = curs.fetchall()
                data = pd.DataFrame(data)
                print(data)
            except Exception as e:
                logging.debug('Update Query')
            # data = pd.read_sql(query, conn, **kwargs)
            conn.commit()
            conn.close()
        except exc.ResourceClosedError:
            return True
        except:
            logging.exception(f'Something went wrong while executing query. Check trace.')
            params = kwargs['params'] if 'params' in kwargs else None
            return False

        return data.where((pd.notnull(data)), None).set_index('id')


    def get_all(self, table, database=None, discard=None):
        """
        Get all data from an SQL table.
        Args:
            table (str): Name of the table from which data should be extracted.
            database (str): Name of the database in which the table lies. Leave
                it none if you want use database during object creation.
            discard (list): columns to be excluded while selecting all
        Returns:
            (DataFrame) A pandas dataframe containing the data. (None if an error
            occurs)
        """
        logging.info(f'Getting all data from {table}')
        if discard:
            logging.info(f'Discarding columns {discard}')
            columns = list(self.execute_default_index(f'SHOW COLUMNS FROM {table}',database).Field)
            columns = [col for col in columns if col not in discard]
            columns_str = json.dumps(columns).replace("'",'').replace('"','')[1:-1]
            return self.execute(f'SELECT {columns_str} FROM {table}', database)

        return self.execute(f'SELECT * FROM {table}', database)

    def get_latest(self, data, group_by_col, sort_col):
        """
        Group data by a column containing repeated values and get latest from it by
        taking the latest value based on another column.
        Example:
        Get the latest products
            id     product   date
            220    6647     2014-09-01
            220    6647     2014-10-16
            826    3380     2014-11-11
            826    3380     2015-05-19
            901    4555     2014-09-01
            901    4555     2014-11-01
        The function will return
            id     product   date
            220    6647     2014-10-16
            826    3380     2015-05-19
            901    4555     2014-11-01
        Args:
            data (DataFrame): Pandas DataFrame to query on.
            group_by_col (str): Column containing repeated values.
            sort_col (str): Column to identify the latest record.
        Returns:
            (DataFrame) Contains the latest records. (None if an error occurs)
        """
        try:
            logging.info('Grouping data...')
            logging.info(f'Data: {data}')
            logging.info(f'Group by column: {group_by_col}')
            logging.info(f'Sort column: {sort_col}')
            return data.sort_values(sort_col).set_index('id').groupby(group_by_col).tail(1)
        except KeyError as e:
            logging.error(f'Column {e.args[0]} does not exist.')
            return None
        except:
            logging.exception('Something went wrong while grouping data.')
            return None
        
        
import os
os.environ['HOST_IP'] = '13.233.100.20'
os.environ['LOCAL_DB_USER'] = 'sa'
os.environ['LOCAL_DB_PASSWORD'] = 'Akhil@Akhil1'
os.environ['LOCAL_DB_PORT'] = '3306'

tenant_id = 'karvy'

db_config = {
    'host': os.environ['HOST_IP'],
    'user': os.environ['LOCAL_DB_USER'],
    'password': os.environ['LOCAL_DB_PASSWORD'],
    'port': os.environ['LOCAL_DB_PORT']
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from apply_business_rule import apply_business_rule
case_id = None
function_params = {'stage':['ONLINE_OPERATIONS']}
tenant_id = 'karvy'
file_path = f"D:\\Desktop\\Karvy\\Feeds_Input_April\\ONLINE\\Operations\\MIRAEASSET_AllBanks_20200428164607.xls"
look_ups = {'holiday1':"",
           'holiday2':""}
r = apply_business_rule(case_id, function_params, tenant_id, file_path)

INFO:root:
 case_id None 
function_params {'stage': ['ONLINE_OPERATIONS']} 
tenant_id karvy

INFO:root:
 got the stage ONLINE_OPERATIONS 

INFO:root:
 Had to apply the column rules because column bool is ONLINE_OPERATIONS 

INFO:root:Host: 13.233.100.20
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Before converting: SELECT * from [sequence_rule_data] where [group] = %s
DEBUG:root:After converting: SELECT * from [sequence_rule_data] where [group] = ?
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_business_rules. [<pyodbc.Connection object at 0x0000027E5F362AC0>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000027E5F7BD378>]
DEBUG:root:Params: ('ONLINE_OPERATIONS',)
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Columns: ['id', 'rule_string', 'description', 'rule_id', 'next_if_sucess', 'next_if_failure', '

DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000027E5F6C7B70>]
DEBUG:root:Params: ('ONLINE_OPERATIONS',)
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Columns: ['id', 'Name', 'type_', 'folder_name', 'sub_feed', 'headers', 'required_columns', 'required_column_type_mapping', 'date_column_format', 'raw_column_mapping', 'standard_column_mapping', 'table_mapping', 'duplicates', 'sheetname', 'account_number_match', 'Trans_descp_map', 'account_subfeed_map', 'default_values', 'row_count', 'dict_column_split_value', 'assign_basedon_compare', 'ffill_columns']
INFO:root:Got data from preprocessing master is :      id    Name type_        folder_name           sub_feed  \
0  36.0  ONLINE  feed  ONLINE_OPERATIONS  online_operations   

                                             headers  \
0  ["Biller Id", "Bank Id", "Bank Ref. No.", "PGI...   

                                    required_columns  \
0  ["Biller Id", "

INFO:root:successfully assigned default values
INFO:root:ffill_values data is {}
INFO:root:Got type conversion data is : {'Ref. 1': 'int64', 'Ref. 2': 'int64', 'Ref. 5': 'int64', 'Ref. 8': 'int64'}
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'TSD1A118015251'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'TSD1A118015251'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'TSD1A118015210'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'MCD1A118007297'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1A118006297'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'OND1A118006296'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD1A118005569'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() w

ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1A118006085'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'CFD1A118006018'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'PFD1G118005886'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1A118005813'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD2A118005642'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1A118005568'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD2A118005383'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'CFD1G118005372'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD1A118005360'
ERROR:root:error in type conversion
ERROR:root:invalid literal f

ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD1A117978644'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD1A117978103'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD1A117978102'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'HCD1A117977505'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1G117976869'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1A117976867'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'IOD1A118015552'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'PFD1A118015353'
ERROR:root:error in type conversion
ERROR:root:invalid literal for int() with base 10: 'DBDGA118015089'
ERROR:root:error in type conversion
ERROR:root:invalid literal f

...

0      TSD1
1      TSD1
2      HCRG
3      HCRG
4      MCD1
       ... 
493    CFD1
494    CFD1
495    DBDG
496     NaN
497     NaN
Name: Ref. 7, Length: 498, dtype: object


INFO:root:Host: 13.233.100.20
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_extraction
DEBUG:root:Before converting: INSERT INTO [progress_bar](file_name, stage, folder_name, description, unique_id) VALUES ('MIRAEASSET_AllBanks_20200428164607','Data Manipulation','ONLINE_OPERATIONS','Data manipulation rules applied successfully', 'None')
DEBUG:root:After converting: INSERT INTO [progress_bar](file_name, stage, folder_name, description, unique_id) VALUES ('MIRAEASSET_AllBanks_20200428164607','Data Manipulation','ONLINE_OPERATIONS','Data manipulation rules applied successfully', 'None')
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_extraction. [<pyodbc.Connection object at 0x0000027E5F362AC0>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000027E5FA8F120>]
DEBUG:root:Params: ()
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Update Q

required standard mapping got is
{'Feed_ID': 'Feed_ID', 'Feed': 'Feed', 'Sub_Feed': 'Sub_Feed', 'Date': 'Date', 'Bank Ref. No.': 'Code', 'Filter': 'Filter', 'Amount(Rs.Ps)': 'Amount', 'Matched': 'Matched', 'ID': 'ID', 'queue': 'queue', 'Credit Received Date': 'Credit Received Date', 'Credit Received Time': 'Credit Received Time'}


INFO:root:Host: 13.233.100.20
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Executing `execute` instead of `execute_default_index`
DEBUG:root:Before converting: UPDATE [preprocessing_master] SET [row_count] = %s WHERE [sub_feed] = %s
DEBUG:root:After converting: UPDATE [preprocessing_master] SET [row_count] = ? WHERE [sub_feed] = ?
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_business_rules. [<pyodbc.Connection object at 0x0000027E5F362AC0>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000027E5FA8F738>]
DEBUG:root:Params: (17019.0, 'ONLINE_OPERATIONS')
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Update Query
DEBUG:root:Data is not a DataFrame. Returning True. [<class 'NoneType'>]
INFO:root:done generating feed
DEBUG:root:got type is : feed
DEBUG:root:Required raw mapping is : {'Feed_ID': 'Feed_ID', '

DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_extraction. [<pyodbc.Connection object at 0x0000027E5F362AC0>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000027E5FA82A80>]
DEBUG:root:Params: ()
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Update Query
DEBUG:root:Data is not a DataFrame. Returning True. [<class 'NoneType'>]


In [12]:
r

{'flag': True,
 'message': 'Applied business rules columnwise successfully.',
 'produce_data': {'standard_processed_file_path': 'D:\\Desktop\\Karvy\\Feeds_Input_April\\ONLINE\\Operations\\MIRAEASSET_AllBanks_20200428164607_standard_processed_1588765473.csv',
  'raw_processed_file_path': 'D:\\Desktop\\Karvy\\Feeds_Input_April\\ONLINE\\Operations\\MIRAEASSET_AllBanks_20200428164607_raw_processed_1588765473.csv',
  'rejected_file_path': 'D:\\Desktop\\Karvy\\Feeds_Input_April\\ONLINE\\Operations\\MIRAEASSET_AllBanks_20200428164607_rejected_processed_1588765473.csv',
  'stage': 'ONLINE_OPERATIONS',
  'type': 'feed'},
 'output': <BusinessRules.BusinessRules at 0x27e5f980c88>}

In [ ]:
# for raw_mapping db:df
# for standard_mapping df:db

In [7]:
import pandas as pd
sample_dt = {"s.no":["1","2"],"transfer_notes":['{"Fund":"117","Ihno":"116252098","Scheme":"TS  ","Plan":"RG  ","PaymentId":"pay_ERCG9LGkMkA2HY"}','{"Fund":"117","Ihno":"116269427","Scheme":"IO  ","Plan":"RG  ","PaymentId":"pay_ERFpjJVf7Ii15p"}']}
df = pd.DataFrame(sample_dt)
df['Scheme'] = ''
df['Ihno'] = ''
df
df['transfer_notes'] = df['transfer_notes'].apply(lambda x: json.loads(x))

In [2]:
df

,s.no,transfer_notes,Scheme,Ihno
0,1,"{""Fund"":""117"",""Ihno"":""116252098"",""Scheme"":""TS ...",,
1,2,"{""Fund"":""117"",""Ihno"":""116269427"",""Scheme"":""IO ...",,


In [8]:
import pandas as pd
import json

a = {"json_column": "transfer_notes",
 "output_column_key": {"Scheme":"Scheme", "Ihno":'Ihno'}
}

sample_df = df
col2_expanded = sample_df[a["json_column"]].apply(lambda x:pd.Series(x))
print(col2_expanded)
sample_df[list(a["output_column_key"].keys())] = col2_expanded[list(a["output_column_key"].values())]
sample_df

  Fund       Ihno Scheme  Plan           PaymentId
0  117  116252098   TS    RG    pay_ERCG9LGkMkA2HY
1  117  116269427   IO    RG    pay_ERFpjJVf7Ii15p


,s.no,transfer_notes,Scheme,Ihno
0,1,"{'Fund': '117', 'Ihno': '116252098', 'Scheme':...",TS,116252098
1,2,"{'Fund': '117', 'Ihno': '116269427', 'Scheme':...",IO,116269427


In [7]:
# get raw_dataframe column name
import json
db = DB('business_rules', tenant_id=tenant_id, **db_config)
map_stage = 'CPVIRTUAL_AK99'
map_query = f"SELECT * FROM [standard_column_mapping] WHERE [feed_subfeed] = '{map_stage}'"
map_ = db.execute_default_index(map_query,'karvy_business_rules')
map_ = json.loads(map_['raw_column_mapping'][0])

raw_mapping = {v:k for k,v in map_.items()}

raw_mapping = json.dumps(raw_mapping)
map_ = list(map_.values())[3:-3]
map_ = json.dumps(map_)

headers = map_

required_columns = map_

raw_column_mapping = raw_mapping


print(headers)
print('\n')
print(required_columns)
print('\n')
print(raw_column_mapping)

INFO:root:Host: 18.232.109.233
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Executing `execute` instead of `execute_default_index`
DEBUG:root:Before converting: SELECT * FROM [standard_column_mapping] WHERE [feed_subfeed] = 'CPVIRTUAL_AK99'
DEBUG:root:After converting: SELECT * FROM [standard_column_mapping] WHERE [feed_subfeed] = 'CPVIRTUAL_AK99'
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_business_rules. [<pyodbc.Connection object at 0x000001CC1252E920>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x000001CC101DD288>]
DEBUG:root:Params: ()
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Columns: ['id', 'feed_subfeed', 'mapping', 'row_count', 'date_columns', 'raw_column_mapping', 'string_columns', 'date_empty_columns', 'drop_columns', 'header_type_mapping', 'strip_columns', 'required_col_list']


["Channel Partner", "AMC_CODE", "Scheme Code", "Scheme Name", "Tran id", "Sub Transaction Type", "Investor Name", "Account No", "Amount", "Pmt Mode", "Trxn Date", "Credit Date", "Transaction Reference Number"]


["Channel Partner", "AMC_CODE", "Scheme Code", "Scheme Name", "Tran id", "Sub Transaction Type", "Investor Name", "Account No", "Amount", "Pmt Mode", "Trxn Date", "Credit Date", "Transaction Reference Number"]


{"Feed_ID": "Feed_ID", "Raw_ID": "Raw_ID", "case_id": "case_id", "Channel Partner": "Channel Partner", "AMC_CODE": "AMC_CODE", "Scheme Code": "Scheme Code", "Scheme Name": "Scheme Name", "Tran id": "Tran id", "Sub Transaction Type": "Sub Transaction Type", "Investor Name": "Investor Name", "Account No": "Account No", "Amount": "Amount", "Pmt Mode": "Pmt Mode", "Trxn Date": "Trxn Date", "Credit Date": "Credit Date", "Transaction Reference Number": "Transaction Reference Number", "Filter": "Filter", "Matched": "Matched", "queue": "queue"}


In [13]:
# get raw_dataframe column name
import json

map_ = {"Feed_ID":"Feed_ID", "Raw_ID":"Raw_ID", "case_id":"case_id", "ACCOUNTNUMBER": "ACCOUNTNUMBER",  "CONSOLIDATION_NMBR": "CONSOLIDATION_NMBR",  "TRANS_DATE": "TRANS_DATE",  "Scheme_Code": "CLIENT_CODE",  "CLIENT_CODE": "Scheme Code",  "ROW_TYPE": "ROW_TYPE",  "ENTRY_ID": "ENTRY_ID",  "TYPE_OF_EN": "TYPE_OF_EN",  "DR_CR": "DR_CR",  "ENTRY_AMT": "ENTRY_AMT",  "VAL_DT": "VAL_DT",  "POST_DT": "POST_DT",  "PROD_CODE": "PROD_CODE",  "PKUP_LOC": "PKUP_LOC",  "PKUP_PT": "PKUP_PT",  "PKUP_DT": "PKUP_DT",  "DEPT_SLIP": "DEPT_SLIP",  "DEPT_DT": "DEPT_DT",  "DEPT_AMT": "DEPT_AMT",  "NO_OF_INST": "NO_OF_INST",  "DEPT_RMK": "DEPT_RMK",  "INST_NO": "INST_NO",  "DRAWEE_BK": "DRAWEE_BK",  "CL_LOC": "CL_LOC",  "INST_AMT": "INST_AMT",  "INST_DT": "INST_DT",  "DRAWER_NAM": "DRAWER_NAM",  "RT_REASON": "RT_REASON",  "LIQUIDATION_NMBR": "LIQUIDATION_NMBR",  "LIQUIDATION_DATE": "LIQUIDATION_DATE",  
        "Filter": "Filter",  "Matched": "Matched",  "IN_HOUSE_NO":"IN_HOUSE_NO", "queue":"queue"}
raw_mapping = {v:k for k,v in map_.items()}

raw_mapping = json.dumps(raw_mapping)
map_ = list(map_.values())[3:-3]
map_ = json.dumps(map_)

headers = map_

required_columns = map_

raw_column_mapping = raw_mapping


print(headers)
print('\n')
print(required_columns)
print('\n')
print(raw_column_mapping)

["ACCOUNTNUMBER", "CONSOLIDATION_NMBR", "TRANS_DATE", "CLIENT_CODE", "Scheme Code", "ROW_TYPE", "ENTRY_ID", "TYPE_OF_EN", "DR_CR", "ENTRY_AMT", "VAL_DT", "POST_DT", "PROD_CODE", "PKUP_LOC", "PKUP_PT", "PKUP_DT", "DEPT_SLIP", "DEPT_DT", "DEPT_AMT", "NO_OF_INST", "DEPT_RMK", "INST_NO", "DRAWEE_BK", "CL_LOC", "INST_AMT", "INST_DT", "DRAWER_NAM", "RT_REASON", "LIQUIDATION_NMBR", "LIQUIDATION_DATE", "Filter"]


["ACCOUNTNUMBER", "CONSOLIDATION_NMBR", "TRANS_DATE", "CLIENT_CODE", "Scheme Code", "ROW_TYPE", "ENTRY_ID", "TYPE_OF_EN", "DR_CR", "ENTRY_AMT", "VAL_DT", "POST_DT", "PROD_CODE", "PKUP_LOC", "PKUP_PT", "PKUP_DT", "DEPT_SLIP", "DEPT_DT", "DEPT_AMT", "NO_OF_INST", "DEPT_RMK", "INST_NO", "DRAWEE_BK", "CL_LOC", "INST_AMT", "INST_DT", "DRAWER_NAM", "RT_REASON", "LIQUIDATION_NMBR", "LIQUIDATION_DATE", "Filter"]


{"Feed_ID": "Feed_ID", "Raw_ID": "Raw_ID", "case_id": "case_id", "ACCOUNTNUMBER": "ACCOUNTNUMBER", "CONSOLIDATION_NMBR": "CONSOLIDATION_NMBR", "TRANS_DATE": "TRANS_DATE", "CLIENT_C

In [17]:
# get std_dataframe column name
db = DB('business_rules', tenant_id=tenant_id, **db_config)
map_stage = 'CPVIRTUAL_AK99'
map_query = f"SELECT [mapping] FROM [standard_column_mapping] WHERE [feed_subfeed] = '{map_stage}'"
map_ = db.execute_default_index(map_query,'karvy_business_rules')
map_ = json.loads(map_['mapping'][0])
print(list(map_.keys()))

INFO:root:Host: 18.232.109.233
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Executing `execute` instead of `execute_default_index`
DEBUG:root:Before converting: SELECT [mapping] FROM [standard_column_mapping] WHERE [feed_subfeed] = 'CPVIRTUAL_AK99'
DEBUG:root:After converting: SELECT [mapping] FROM [standard_column_mapping] WHERE [feed_subfeed] = 'CPVIRTUAL_AK99'
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_business_rules. [<pyodbc.Connection object at 0x0000021AADD2D850>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000021AAA999AF8>]
DEBUG:root:Params: ()
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Columns: ['mapping']


['Feed_ID', 'Feed', 'Sub_Feed', 'Credit Date', 'Transaction Reference Number', 'Filter', 'Amount', 'Matched', 'ID', 'queue', 'Credit Received Date', 'Credit Received Time']


In [38]:
db = DB('business_rules', tenant_id=tenant_id, **db_config)
map_query = f"INSERT INTO [karvy_business_rules].[dbo].[preprocessing_master] (id, Name, type_, folder_name, sub_feed, headers, required_columns, required_column_type_mapping, date_column_format, raw_column_mapping, standard_column_mapping, table_mapping, duplicates, sheetname, account_number_match, Trans_descp_map, account_subfeed_map, default_values, row_count) VALUES (2, 'CPVIRTUAL', 'feed', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)"
map_ = db.execute_default_index(map_query,'karvy_business_rules')
#map_ = json.loads(map_['mapping'][0])

INFO:root:Host: 18.232.109.233
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Executing `execute` instead of `execute_default_index`
DEBUG:root:Before converting: INSERT INTO [karvy_business_rules].[dbo].[preprocessing_master] (id, Name, type_, folder_name, sub_feed, headers, required_columns, required_column_type_mapping, date_column_format, raw_column_mapping, standard_column_mapping, table_mapping, duplicates, sheetname, account_number_match, Trans_descp_map, account_subfeed_map, default_values, row_count) VALUES (2, 'CPVIRTUAL', 'feed', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
DEBUG:root:After converting: INSERT INTO [karvy_business_rules].[dbo].[preprocessing_master] (id, Name, type_, folder_name, sub_feed, headers, required_columns, required_column_type_mapping, date_column_format, raw_column_mapping, standard_column_mapping, table_mapping, duplicates, sheetname, account_number_match, Trans_descp

In [9]:
L1 = ['Account Number', 'Value Date', 'DR / CR', 'Transaction Amount',
       'Account Owner Reference', 'Servicing Institution Reference',
       'Information to Account Owner', 'Transaction Description',
       'Transaction Post Time', 'Filter', 'Matched', 'queue', 'ID', 'Code',
       'matched_amount', 'time_stamp', 'Credit Received Date',
       'Credit Received Time', 'Feed', 'Scheme', 'Sub_Feed', 'Bank_ID',
       'Raw_ID', 'case_id']

In [10]:
d = dict(zip(L1,L1))
d

{'Account Number': 'Account Number',
 'Value Date': 'Value Date',
 'DR / CR': 'DR / CR',
 'Transaction Amount': 'Transaction Amount',
 'Account Owner Reference': 'Account Owner Reference',
 'Servicing Institution Reference': 'Servicing Institution Reference',
 'Information to Account Owner': 'Information to Account Owner',
 'Transaction Description': 'Transaction Description',
 'Transaction Post Time': 'Transaction Post Time',
 'Filter': 'Filter',
 'Matched': 'Matched',
 'queue': 'queue',
 'ID': 'ID',
 'Code': 'Code',
 'matched_amount': 'matched_amount',
 'time_stamp': 'time_stamp',
 'Credit Received Date': 'Credit Received Date',
 'Credit Received Time': 'Credit Received Time',
 'Feed': 'Feed',
 'Scheme': 'Scheme',
 'Sub_Feed': 'Sub_Feed',
 'Bank_ID': 'Bank_ID',
 'Raw_ID': 'Raw_ID',
 'case_id': 'case_id'}

In [191]:
['TXN No', 'Transaction Date', 'Investor Name', 'Scheme Name', 'Scheme Code', 'Amount', 'UTR Number']

In [199]:
import pandas as pd
path = 'D:\\Desktop\\Karvy_Rules_Statuses\\inputs\\CPVIRTUAL_AZ99\\07-06-2019_Mirae_Regular_MIS.csv'
df = pd.read_csv(path)
df

,TXN No,Transaction Date,Investor Name,Scheme Name,Scheme Code,Amount,UTR Number
0,477653.0,06-03-2019 00:00,BNJU BHBSXBM,Mirae Asset Large Cap Fund Growth,IORGG,7651.0,ICICR22019060400486781
1,479124.0,06-02-2019 00:00,TBOEFFQ SBHIBWFOESB SBP,Mirae Asset Large Cap Fund Growth,IORGG,42487.0,CMS1140363877
2,479146.0,06-03-2019 00:00,TBOEFFQ SBHIBWFOESB SBP,Mirae Asset Large Cap Fund Growth,IORGG,15181.0,CMS1140363877
3,480123.0,06-03-2019 00:00,Tbuqbm Tibsnb,Mirae Asset Large Cap Fund Growth,IORGG,13551.0,ICICR22019060400486781
4,480134.0,06-03-2019 00:00,BWOFFU TJOHI CJOESB,Mirae Asset Large Cap Fund Growth,IORGG,8280.0,ICICR22019060400486781
...,...,...,...,...,...,...,...
908,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
911,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
#df = df.drop(drop_columns, axis=1, inplace=False)
df2 = df[drop_columns]
df2

,Amount,Scheme Code,TXN No
0,7651.0,IORGG,477653.0
1,42487.0,IORGG,479124.0
2,15181.0,IORGG,479146.0
3,13551.0,IORGG,480123.0
4,8280.0,IORGG,480134.0
...,...,...,...
908,NaN,NaN,NaN
909,NaN,NaN,NaN
910,NaN,NaN,NaN
911,NaN,NaN,NaN


In [201]:
import json
import logging
def select_req_columns_df(df, stage, tenant_id):
    # selecting a dataframe with required columns 
    try:
        req_col_df_query = "SELECT [required_col_list] FROM [standard_column_mapping] where [feed_subfeed] = %s"
        business_rules_db = DB('business_rules', tenant_id=tenant_id, **db_config)
        req_cols_list_df = business_rules_db.execute_default_index(req_col_df_query, params=[stage])

        req_cols_list = json.loads(req_cols_list_df['required_col_list'][0])
        logging.info(f'input dataframe columns are ===>',list(df.columns))
        logging.info(f'required dataframe columns are ===>',req_cols_list)
        req_df = df[req_cols_list]
        return req_df
    except Exception as e:
        logging.error(f'required column names are not exists in dataframe')
        logging.error(e)
        req_col_df_query = "SELECT [required_col_list] FROM [standard_column_mapping] where [feed_subfeed] = %s"
        business_rules_db = DB('business_rules', tenant_id=tenant_id, **db_config)
        req_cols_list_df = business_rules_db.execute_default_index(req_col_df_query, params=[stage])

        req_cols_list = json.loads(req_cols_list_df['required_col_list'][0])
        req_df = df[req_cols_list]
        return req_df

df = select_req_columns_df(df,'CPVIRTUAL_AZ99',tenant_id)
df

INFO:root:Host: 18.232.109.233
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Executing `execute` instead of `execute_default_index`
DEBUG:root:Before converting: SELECT [required_col_list] FROM [standard_column_mapping] where [feed_subfeed] = %s
DEBUG:root:After converting: SELECT [required_col_list] FROM [standard_column_mapping] where [feed_subfeed] = ?
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_business_rules. [<pyodbc.Connection object at 0x0000022F5B7B0E00>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000022F5C35A828>]
DEBUG:root:Params: ('CPVIRTUAL_AZ99',)
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Columns: ['required_col_list']


input dataframe columns are ===> ['TXN No', 'Transaction Date', 'Investor Name', 'Scheme Name', 'Scheme Code', 'Amount', 'UTR Number']
required dataframe columns are ===> ['TXN No', 'Transaction Date', 'Investor Name', 'Amount', 'UTR Number']


,TXN No,Transaction Date,Investor Name,Amount,UTR Number
0,477653.0,06-03-2019 00:00,BNJU BHBSXBM,7651.0,ICICR22019060400486781
1,479124.0,06-02-2019 00:00,TBOEFFQ SBHIBWFOESB SBP,42487.0,CMS1140363877
2,479146.0,06-03-2019 00:00,TBOEFFQ SBHIBWFOESB SBP,15181.0,CMS1140363877
3,480123.0,06-03-2019 00:00,Tbuqbm Tibsnb,13551.0,ICICR22019060400486781
4,480134.0,06-03-2019 00:00,BWOFFU TJOHI CJOESB,8280.0,ICICR22019060400486781
...,...,...,...,...,...
908,NaN,NaN,NaN,NaN,NaN
909,NaN,NaN,NaN,NaN,NaN
910,NaN,NaN,NaN,NaN,NaN
911,NaN,NaN,NaN,NaN,NaN


In [ ]:
map_ = 
map_2 = {v:k for k,v in map_.items()}
map_2

In [38]:
db = DB('business_rules', tenant_id=tenant_id, **db_config)
map_stage = 'CPVIRTUAL_AZ99'
map_query = f"SELECT * FROM [preprocessing_master] WHERE [sub_feed] = '{map_stage}'"
map_ = db.execute_default_index(map_query,'karvy_business_rules')
map_

INFO:root:Host: 18.232.109.233
INFO:root:User: sa
INFO:root:Password: Akhil@Akhil1
INFO:root:Port: 3306
INFO:root:Database: karvy_business_rules
DEBUG:root:Executing `execute` instead of `execute_default_index`
DEBUG:root:Before converting: SELECT * FROM [preprocessing_master] WHERE [sub_feed] = 'CPVIRTUAL_AZ99'
DEBUG:root:After converting: SELECT * FROM [preprocessing_master] WHERE [sub_feed] = 'CPVIRTUAL_AZ99'
DEBUG:root:Connecting to DB
DEBUG:root:Connection established with karvy_business_rules. [<pyodbc.Connection object at 0x0000021958628780>]
DEBUG:root:Cursor object created. [<pyodbc.Cursor object at 0x0000021959497738>]
DEBUG:root:Params: ()
DEBUG:root:Params Type: <class 'tuple'>
DEBUG:root:Query executed.
DEBUG:root:Fetching all data.
DEBUG:root:Columns: ['id', 'Name', 'type_', 'folder_name', 'sub_feed', 'headers', 'required_columns', 'required_column_type_mapping', 'date_column_format', 'raw_column_mapping', 'standard_column_mapping', 'table_mapping', 'duplicates', 'sheetna

,id,Name,type_,folder_name,sub_feed,headers,required_columns,required_column_type_mapping,date_column_format,raw_column_mapping,standard_column_mapping,table_mapping,duplicates,sheetname,account_number_match,Trans_descp_map,account_subfeed_map
0,1,CPVIRTUAL,feed,CPVIRTUAL_AZ99,cpvirtual_az99,"[""TXN No"", ""Transaction Date"", ""Investor Name""...","[""TXN No"", ""Transaction Date"", ""Investor Name""...",{},"{""Transaction Date "":""yyyy-mm-dd""}","{""Feed_ID"": ""Feed_ID"",\n""Raw_ID"": ""Raw_ID"",\n""...","{\n ""Feed_ID"": ""Feed_ID"",\n ""Feed"": ""Feed"",\...",CPVIRTUAL_AZ99,[],None,{},"{""Alphafront"":""CPVIRTUAL_AZ99"",""AZ99"":""CPVIRTU...",[]


In [43]:
x = '["TXN No", "Transaction Date", "Investor Name", "Scheme Name", "Scheme Code", "Amount", "UTR Number"]'
print(x[48])
import json
print(json.loads(x))

"
['TXN No', 'Transaction Date', 'Investor Name', 'Scheme Name', 'Scheme Code', 'Amount', 'UTR Number']


In [46]:
import pandas as pd
bank_data = pd.read_csv("D:\\Desktop\\Karvy\\bank_data.csv")
feed_data = pd.read_csv("D:\\Desktop\\Karvy\\feed_data.csv")
matched = pd.read_csv("D:\\Desktop\\Karvy\\matched.csv")

In [47]:
bank_data

,Unnamed: 0,Feed,SubFeed,Description,Total,MatchedAmount,UnmatchedAmount,Matched,match_type,feed_index,queue,bank_index
0,0,MFU,MFU,2,40,0,40.0,0,no_match,NaN,NaN,0
1,1,MFU,MFU,2,60,0,60.0,0,no_match,NaN,NaN,1
2,2,MFU,MFU,3,200,0,200.0,0,no_match,NaN,NaN,2


In [48]:
Matched_amount = Matched_amount+diff
UnmatchedAmount = UnmatchedAmount - total in matched table
if len(value) > 1 ....first one has to be above else 0 0 in matched amount and unmatched

SyntaxError: invalid syntax (<ipython-input-48-b9e4b7e02da7>, line 2)

In [49]:
feed_data

,Unnamed: 0,BTRN,Amount,Matched,Credit Received Date,Credit Received Time,match_type,CreditReceivedDate,CreditReceivedTime,queue,ID,bank_index,feed_index
0,0,2,10,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,0
1,1,2,10,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,1
2,2,2,10,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,2
3,3,2,20,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,3
4,4,2,30,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,4
5,5,2,20,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,5
6,6,3,10,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,6
7,7,3,20,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,7
8,8,3,170,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,8
9,9,4,500,0,NaN,NaN,no_match,NaN,NaN,NaN,NaN,NaN,9


In [50]:
matched

,Unnamed: 0,Description,bank_index,Total,BTRN,feed_index,Amount,difference,match_type
0,0,2,"(0, 1)",100,2,"(0, 1, 2, 3, )",100,0,exact_match
1,1,3,"(2,)",200,3,"(6, 7, 8)",200,0,exact_match
2,2,4,"(3,)",300,3,"(9, 10, 11)",300,100,partial_match
3,3,5,"(4,5)",400,4,"(12, 13, 14)",400,50,partial_match
4,4,6,"(6,)",500,2,"(4, 5)",500,60,partial_match


In [51]:
extraction_db = DB('extraction', tenant_id=tenant_id, **db_config)

In [52]:
try:
    for index,row in matched.iterrows():
        
        if row['match_type'] == "exact_match":
            value = eval(row['bank_index'])
            feed_index = eval(row['feed_index'])
            if len(value) <= 1 :
                value = '('+str(value[0])+')'
            if len(feed_index) <= 1 :
                feed_index = '('+str(feed_index[0])+')'
            concat_bank_index_query = f"UPDATE `feed_data` SET `bank_index` = CONCAT(`bank_index`,'{value}'+','),`match_type` = 'exact_match' WHERE `feed_index` in {feed_index}"
            print(f"query is {concat_bank_index_query}")
            extraction_db.execute_(concat_bank_index_query)
            concat_feed_index_query = f"UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'{feed_index}'+','), `match_type` = 'exact_match', `MatchedAmount` = `MatchedAmount` + `UnmatchedAmount` ,`UnmatchedAmount` = 0 WHERE `bank_index` in {value}"
            print(f"query is {concat_feed_index_query}")
            extraction_db.execute_(concat_feed_index_query)
            
        if row['match_type'] == "partial_match":
            
            value3 = eval(row['bank_index'])
            value2 = eval(row['bank_index'])
            feed_index3 = eval(row['feed_index'])
            feed_index2 = eval(row['feed_index'])
            print(f"====>",value3)
            if len(value3) <= 1 :
                value2 = '('+str(value3[0])+')'
            if len(feed_index3) <= 1 :
                feed_index2 = '('+str(feed_index3[0])+')'
            # for updating feed_data table  
            concat_bank_index_query = f"UPDATE `feed_data` SET `bank_index` = CONCAT(`bank_index`,'{value2}'+','),`match_type` = 'partial_match' WHERE `feed_index` in {feed_index2}"
            print(f"query is {concat_bank_index_query}")
            extraction_db.execute_(concat_bank_index_query)
            
            # for updating feed_data table
            diff_amount = row['difference']
            Total_amt = row['Total']
            if len(value3) <= 1 :
                print(f"<====",len(value3))
                concat_feed_index_query = f"UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'{feed_index2}'+','), `match_type` = 'partial_match', `MatchedAmount` = `MatchedAmount` + {diff_amount} ,`UnmatchedAmount` = `UnmatchedAmount`- {Total_amt} WHERE `bank_index` in {value2}"
                print(f"query is {concat_feed_index_query}")
                extraction_db.execute_(concat_feed_index_query)
            else:
                print(f"<==++==",value3)
                concat_feed_index_query1 = f"UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'{feed_index2}'+','), `match_type` = 'partial_match', `MatchedAmount` = `MatchedAmount` + {diff_amount} ,`UnmatchedAmount` = `UnmatchedAmount`- {Total_amt} WHERE `bank_index` = {value3[0]}"
                print(f"query is {concat_feed_index_query1}")
                other_values = (value2[1:])
                if len(other_values) <= 1 :
                    other_values = '('+str(value3[0])+')'
                concat_feed_index_query2 = f"UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'{feed_index2}'+','), `match_type` = 'partial_match', `MatchedAmount` = 0 ,`UnmatchedAmount` = 0 WHERE `bank_index` in {other_values}"
                print(f"query is {concat_feed_index_query2}")
                extraction_db.execute_(concat_feed_index_query1)
                extraction_db.execute_(concat_feed_index_query2)
except Exception as e:
    print(f"error is {e}")

query is UPDATE `feed_data` SET `bank_index` = CONCAT(`bank_index`,'(0, 1)'+','),`match_type` = 'exact_match' WHERE `feed_index` in (0, 1, 2, 3)
query is UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'(0, 1, 2, 3)'+','), `match_type` = 'exact_match', `MatchedAmount` = `MatchedAmount` + `UnmatchedAmount` ,`UnmatchedAmount` = 0 WHERE `bank_index` in (0, 1)
query is UPDATE `feed_data` SET `bank_index` = CONCAT(`bank_index`,'(2)'+','),`match_type` = 'exact_match' WHERE `feed_index` in (6, 7, 8)
query is UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'(6, 7, 8)'+','), `match_type` = 'exact_match', `MatchedAmount` = `MatchedAmount` + `UnmatchedAmount` ,`UnmatchedAmount` = 0 WHERE `bank_index` in (2)
====> (3,)
query is UPDATE `feed_data` SET `bank_index` = CONCAT(`bank_index`,'(3)'+','),`match_type` = 'partial_match' WHERE `feed_index` in (9, 10, 11)
<==== 1
query is UPDATE `bank_data` SET `feed_index` = CONCAT(`feed_index`,'(9, 10, 11)'+','), `match_type` = 'partial_m

In [53]:
import json
sat = '(2,3,4)'
sat = eval(sat)
if len(sat) <= 1 :
    sat = '('+str(sat[0])+')'
tuple(sat[1:])

(3, 4)

In [ ]:
{
 "AA99": "CPVIRTUAL_FR99",
 "BW99": "CPVIRTUAL_BW99",
 "CHOLAM": "CPVIRTUAL_CL99",
 "CPVIRTUAL_WU99": "CPVIRTUAL_WU99",
 "DC99": "CPVIRTUAL_DC99",
 "DU99": "CPVIRTUAL_DU99",
 "GETCLARITY": "CPVIRTUAL_GT99",
 "ID99": "CPVIRTUAL_ID99",
 "KOTAK": "CPVIRTUAL_KT99",
    "OF99": "CPVIRTUAL_OF99",
,
 "R999": "CPVIRTUAL_R999",
 "RRF": "CPVIRTUAL_RR99",
 "RZ99": "CPVIRTUAL_WB99",
 "Reliance": "CPVIRTUAL_RE99",
 "WEALTH N": "CPVIRTUAL_WN99"
}

In [ ]:
{"CT99": "CPVIRTUAL_CT99"}
{"IA99": "CPVIRTUAL_IA99"}
{"IFAST": "CPVIRTUAL_IF99"}
{"IN99": "CPVIRTUAL_IN99"}
{"IO99": "CPVIRTUAL_IO99"}
{"KY99": "CPVIRTUAL_KY99"}
{"MAF": "CPVIRTUAL_LI99"}
{"LP99": "CPVIRTUAL_LP99"}
{"MONEYBASE": "CPVIRTUAL_MB99"}
{"P999": "CPVIRTUAL_P999"}
{"PC99": "CPVIRTUAL_PC99"}
{"PW99": "CPVIRTUAL_PW99"}
{"SARSA": "CPVIRTUAL_SA99"}
{"RBL": "CPVIRTUAL_TL99"}
{"Virtue": "CPVIRTUAL_VO99"}
{"WA99": "CPVIRTUAL_WA99"}
{"WF99": "CPVIRTUAL_WF99"}
{"Mobikwik": "CPVIRTUAL_R999_ClearFunds"}
{"VF99": "CPVIRTUAL_R999_Invezta"}
{"MoneyFro": "CPVIRTUAL_R999_Moneyfront",
"Moneymap": "CPVIRTUAL_R999_Moneyfront"}

In [ ]:
"Bank_ID": "Bank_ID", "Raw_ID": "Raw_ID", "case_id": "case_id",

In [ ]:
"Filter": "Filter", "Matched": "Matched", "queue": "queue"

In [ ]:
{"Bank_ID": "Bank_ID", "Raw_ID": "Raw_ID", "case_id": "case_id","Account name": "Account name", "Account number": "Account number", "Bank name": "Bank name", "Currency": "Currency", "Location": "Location", "BIC": "BIC", "IBAN": "IBAN", "Account status": "Account status", "Account type": "Account type", "Closing ledger balance": "Closing ledger balance", "Closing ledger brought forward from": "Closing ledger brought forward from", "Closing available balance": "Closing available balance", "Closing available brought forward from": "Closing available brought forward from", "Current ledger balance": "Current ledger balance", "Current ledger as at": "Current ledger as at", "Current available balance": "Current available balance", "Current available as at": "Current available as at", "Bank reference": "Bank reference", "Narrative": "Narrative", "Customer reference": "Customer reference", "TRN type": "TRN type", "Value date": "Value date", "Credit amount": "Credit amount", "Debit amount": "Debit amount", "Balance": "Balance", "Time": "Time", "Post date": "Post date","Filter": "Filter", "Matched": "Matched", "queue": "queue"}

In [ ]:
{"Feed_ID":"Feed_ID", "Feed": "Feed", "Sub_Feed": "Sub_Feed", "Narrative": "Code", "Value date": "Date", "Credit amount": "Amount","Filter": "Filter", "Matched": "Matched", "ID":"ID", "queue":"queue", "Credit Received Date":"Credit Received Date","Credit Received Time":"Credit Received Time"}

In [ ]:
import json

map_ =["Narrative": "Code", "Value date": "Date", "Credit amount": "Amount"]
raw_mapping = {v:k for k,v in map_.items()}

raw_mapping = json.dumps(raw_mapping)
map_ = list(map_.values())[3:-3]
map_ = json.dumps(map_)

headers = map_

required_columns = map_

raw_column_mapping = raw_mapping


print(headers)
print('\n')
print(required_columns)
print('\n')
print(raw_column_mapping)

In [ ]:
from apply_business_rule import apply_business_rule
case_id = None
function_params = {'stage':['CMS']}
tenant_id = 'karvy'
file_path = f"D:\\Desktop\\Karvy\\feed_sample_data\\feed_sample_data\\CMS\\INPUTS AND OUTPUTS\\MIRAE-03Mar2020.xls"
#file_path = f"D:\\Desktop\\Karvy\\feed_sample_data\\feed_sample_data\\MFU\\INPUTS AND OUTPUTS\\200303MAF012-T.xls"
look_ups = {'holiday1':"D:\\Desktop\\Karvy\\feed_sample_data\\feed_sample_data\\CMS\\INPUTS AND OUTPUTS\\MIRAE ASSET MF_03032020064101.xls",
           'holiday2':"D:\\Desktop\\Karvy\\feed_sample_data\\feed_sample_data\\CMS\\INPUTS AND OUTPUTS\\MIRAE ASSET MF HOLIDAY AND DATA ENTRY REJEC_03032020064104.xls‪"}
r = apply_business_rule(case_id, function_params, tenant_id, file_path)